In [10]:
# Scrape ESPN BPI 2025 https://www.espn.com/mens-college-basketball/bpi

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import io
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

y = 2025
driver.get(f'https://www.espn.com/mens-college-basketball/bpi/_/season/{y}')
driver.maximize_window()
time.sleep(10)
#element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler"))).click()

try:
    for _ in range(10):
        show_more_link = driver.find_element(By.CSS_SELECTOR, "a.loadMore__link")
        show_more_link.click()
        time.sleep(5)  # wait for the content to load
except:
    try:
        element = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.XPATH, "//table")))
        with io.StringIO(driver.page_source) as f:
            df = pd.read_html(f)[0]
            df2 = pd.read_html(f)[1]
    finally:
        
        bpi = pd.concat([df, df2], axis=1)
        bpi['Year']=y
        bpi = bpi.iloc[:,[12,0,6,7]]
        bpi.columns = ['Year','Team','BPI(O)','BPI(D)']
        print(y)
espnBPI = pd.read_csv('step02b_espnbpi0824.csv')
espnBPI = espnBPI[espnBPI['Year']<y]   
espnBPI = pd.concat([espnBPI,bpi]) 

espnBPI.to_csv('step02c_espnbpi0825.csv',index=False)      

a = espnBPI['Team'].unique()
b = pd.DataFrame({'tm':sorted(a)})
b.to_csv('bbb.csv',index=False)

2025
